In [1]:
import os
import psycopg
from dotenv import load_dotenv

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

In [2]:
load_dotenv()

True

In [23]:
TABLE_NAME = 'users_churn' # таблица с данными

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'Data_Exploration' # название эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = 'category_numeric_ColumnTransformer' # название зарегистрированной модели 

In [4]:
connection = {'sslmode' : 'require', 'target_session_attrs' : 'read-write'}
postgres_credentials = {
    'host' : os.getenv('DB_DESTINATION_HOST'),
    'port' : os.getenv('DB_DESTINATION_PORT'),
    'user' : os.getenv('DB_DESTINATION_USER'),
    'password' : os.getenv('DB_DESTINATION_PASSWORD'),
    'dbname' : os.getenv('DB_DESTINATION_NAME')
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f'SELECT * FROM {TABLE_NAME}')
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

        df = pd.DataFrame(data, columns=columns)

In [5]:
columns_without_datetime = df.select_dtypes(exclude='datetime').columns
df = df.dropna(subset=columns_without_datetime)

In [6]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
2,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,No,No,No,No,Male,0,No,No,No,1
4,5,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,...,No,No,No,No,Female,0,No,No,No,1
5,6,9305-CDSKC,2019-03-01,2019-11-01,Month-to-month,Yes,Electronic check,99.65,820.50,Fiber optic,...,Yes,No,Yes,Yes,Female,0,No,No,Yes,1
6,7,1452-KIOVK,2018-04-01,NaT,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,Fiber optic,...,No,No,Yes,No,Male,0,No,Yes,Yes,0


In [7]:
obj_df = df.select_dtypes(include="object")

In [8]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(
    categories='auto',
    handle_unknown="ignore", 
    max_categories=10, 
    sparse_output=False, 
    drop='first'
    ) # ваш код здесь #

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy()) # ваш код здесь #

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns)) # ваш код здесь #
encoded_df
# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,partner,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,gender_Male
1,5575-GNVDE,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,...,No,No,No,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3668-QPYBK,Month-to-month,Yes,Mailed check,DSL,Yes,Yes,No,No,No,...,No,No,No,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [9]:
num_df = df[['monthly_charges', 'total_charges']]
num_df

,monthly_charges,total_charges
1,56.95,1889.50
2,53.85,108.15
4,70.70,151.65
5,99.65,820.50
6,89.10,1949.40
...,...,...
7035,78.70,1495.10
7038,84.80,1990.50
7039,103.20,7362.90
7041,74.40,306.60


In [10]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline) # ваш код здесь #
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles) # ваш код здесь #
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler() # ваш код здесь #
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree) # ваш код здесь #
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)
num_df = pd.concat([num_df, encoded_df], axis=1) # ваш код здесь #

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample) # ваш код здесь #
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1) # ваш код здесь #


num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,...,total_charges_robust,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
1,56.95,1889.50,0.010664,0.304005,0.571559,0.113482,0.00029,0.0,0.039206,0.450728,...,-0.532458,2899.8225,5823.8775,11696.4225,156155.441625,313615.803375,6.298524e+05,1.264968e+06,0.0,0.0
2,53.85,108.15,0.000212,0.106306,0.565757,0.315695,0.01203,0.0,0.037628,0.445599,...,-0.522130,4998.4900,10721.6550,22997.7225,353393.243000,758021.008500,1.625939e+06,3.487605e+06,1.0,0.0


In [11]:
cat_columns = ["type", "payment_method", "internet_service", "gender"]
num_columns = ["monthly_charges", "total_charges"]

# Гиперпараметры для трансформеров
n_knots = 3
degree_spline = 4
n_quantiles = 100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoder_rb = RobustScaler()
encoder_pol = PolynomialFeatures(degree=degree)
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)

encoder_oh = OneHotEncoder(
    categories='auto',
    handle_unknown="ignore", 
    max_categories=10, 
    sparse_output=False, 
    drop='first'
    )

In [12]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spline', encoder_spl, num_columns),
        ('quantile', encoder_q, num_columns),
        ('robust', encoder_rb, num_columns),
        ('poly', encoder_pol, num_columns),
        ('kbins', encoder_kbd, num_columns)
    ],
    verbose_feature_names_out=True  # Добавляем префиксы к именам колонок
)


categorical_transformer = Pipeline(
    steps=[
        ('onehot', encoder_oh)
    ]
)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer, cat_columns)
    ],
    verbose_feature_names_out=True  # Автоматическое добавление префиксов
)

In [13]:
encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df.reset_index(drop=True), transformed_df.reset_index(drop=True)], axis=1)

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,num__poly__total_charges^3,num__kbins__monthly_charges,num__kbins__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
0,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,6.745912e+09,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,1.264968e+06,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [18]:
mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')
mlflow.set_registry_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')

In [19]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment)
else:
    experiment_id = experiment.experiment_id

In [27]:
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(
        sk_model=preprocessor,
        artifact_path='column_transformer'
    )

2025/02/05 13:10:41 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [28]:
run_id

'24906d24e7e74b5da4a8b81daca6e18c'